In [12]:
#Used in Tensorflow Model
import numpy as np
from sklearn.naive_bayes import MultinomialNB
import random

#Usde to for Contextualisation and Other NLP Tasks.
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

#Other
import json
import pickle
import os.path

In [13]:
if not os.path.exists('model'):
    print("Processing the Intents.....")
    with open('intents.json') as json_data:
        intents = json.load(json_data)

Processing the Intents.....


In [14]:
if not os.path.exists('model'):
    words = []
    classes = []
    documents = []
    ignore_words = ['?']
    print("Looping through the Intents to Convert them to words, classes, documents and ignore_words.......")
    for intent in intents['intents']:
        for pattern in intent['patterns']:
            # tokenize each word in the sentence
            w = nltk.word_tokenize(pattern)
            # add to our words list
            words.extend(w)
            # add to documents in our corpus
            documents.append((w, intent['tag']))
            # add to our classes list
            if intent['tag'] not in classes:
                classes.append(intent['tag'])

Looping through the Intents to Convert them to words, classes, documents and ignore_words.......


In [15]:
if not os.path.exists('model'):
    print("Stemming, Lowering and Removing Duplicates.......")
    words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
    words = sorted(list(set(words)))

    # remove duplicates
    classes = sorted(list(set(classes)))

    print (len(documents), "documents")
    print (len(classes), "classes", classes)
    print (len(words), "unique stemmed words", words)

Stemming, Lowering and Removing Duplicates.......
27 documents
9 classes ['goodbye', 'greeting', 'hours', 'mopeds', 'opentoday', 'payments', 'rental', 'thanks', 'today']
48 unique stemmed words ["'d", "'s", 'a', 'acceiv', 'anyon', 'ar', 'bye', 'can', 'card', 'cash', 'credit', 'day', 'do', 'doe', 'good', 'goodby', 'hav', 'hello', 'help', 'hi', 'hour', 'how', 'i', 'is', 'kind', 'lat', 'lik', 'mastercard', 'mop', 'of', 'on', 'op', 'rent', 'see', 'tak', 'thank', 'that', 'ther', 'thi', 'to', 'today', 'we', 'what', 'when', 'which', 'work', 'yo', 'you']


In [16]:
if not os.path.exists('model'):
    print("Creating the Data for our Model.....")
    training = []
    output = []

    print("Creating Traning Set, Bag of Words for our Model....")
    for doc in documents:
        # initialize our bag of words
        bag = []
        # list of tokenized words for the pattern
        pattern_words = doc[0]
        # stem each word
        pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
        # create our bag of words array
        for w in words:
            bag.append(1) if w in pattern_words else bag.append(0)
        
        # create a tuple with bag of words and tag regerences.
        training.append([bag, doc[1]])

Creating the Data for our Model.....
Creating Traning Set, Bag of Words for our Model....


In [17]:
if not os.path.exists('model'):
    print("Shuffling Randomly and Converting into Numpy Array for Faster Processing......")
    random.shuffle(training)
    training = np.array(training)

    print("Creating Train and Test Lists.....")
    train_x = list(training[:,0])
    train_y = list(training[:,1])

Shuffling Randomly and Converting into Numpy Array for Faster Processing......
Creating Train and Test Lists.....


/tmp/ipykernel_3999/375135728.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [18]:
if not os.path.exists('model'):
    model = MultinomialNB()

In [19]:
if not os.path.exists('model'):
    print("Training the Model.......")
    model.fit(train_x, train_y)
    print("Saving the Model.......")


Training the Model.......
Saving the Model.......


In [20]:
if not os.path.exists('model'):
    print("Pickle is also Saved..........")
    pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, 
                open( "training_data", "wb" ) )
    pickle.dump(model, open("model", "wb"))

Pickle is also Saved..........


In [21]:
print("Loading Pickle.....")
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']


with open('intents.json') as json_data:
    intents = json.load(json_data)
    
print("Loading the Model......")
# load our saved model
model = pickle.load(open("model", "rb"))

Loading Pickle.....
Loading the Model......


In [35]:
def clean_up_sentence(sentence):
    # It Tokenize or Break it into the constituents parts of Sentense.
    sentence_words = nltk.word_tokenize(sentence)
    # Stemming means to find the root of the word.
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# Return the Array of Bag of Words: True or False and 0 or 1 for each word of bag that exists in the Sentence
def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def response(sentence, userID='123', show_details=False):
    result = model.predict([bow(sentence, words)])
    if result:
        # Long Loop to get the Result.
        for i in intents['intents']:
            # Tag Finding
            if i['tag'] == result:
                # Random Response from High Order Probabilities
                return random.choice(i['responses'])
            
    return 'Repeat Please'

In [36]:
while True:
    input_data = input("You- ")
    answer = response(input_data)
    print(answer)

Hi there, how can I help?
Hi there, how can I help?
Hi there, how can I help?
Our hours are 9am-9pm every day
Our hours are 9am-9pm every day
We're open every day from 9am-9pm
We're open every day from 9am-9pm
Good to see you again
Good to see you again


KeyboardInterrupt: Interrupted by user